In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
df=pd.read_csv("/kaggle/input/crop-yield-and-environmental-factors-2014-2023/crop_yield_dataset.csv")
print(df.head())
print(len(df))

         Date Crop_Type Soil_Type  Soil_pH  Temperature   Humidity  \
0  2014-01-01     Wheat     Peaty     5.50     9.440599  80.000000   
1  2014-01-01      Corn     Loamy     6.50    20.052576  79.947424   
2  2014-01-01      Rice     Peaty     5.50    12.143099  80.000000   
3  2014-01-01    Barley     Sandy     6.75    19.751848  80.000000   
4  2014-01-01   Soybean     Peaty     5.50    16.110395  80.000000   

   Wind_Speed     N     P     K  Crop_Yield  Soil_Quality  
0   10.956707  60.5  45.0  31.5    0.000000     22.833333  
1    8.591577  84.0  66.0  50.0  104.871310     66.666667  
2    7.227751  71.5  54.0  38.5    0.000000     27.333333  
3    2.682683  50.0  40.0  30.0   58.939796     35.000000  
4    7.696070  49.5  45.0  38.5   32.970413     22.166667  
36520


In [2]:
df=df[df['Crop_Yield']!=0.0]
print(len(df))
print(df.head())

25495
         Date Crop_Type Soil_Type  Soil_pH  Temperature   Humidity  \
1  2014-01-01      Corn     Loamy     6.50    20.052576  79.947424   
3  2014-01-01    Barley     Sandy     6.75    19.751848  80.000000   
4  2014-01-01   Soybean     Peaty     5.50    16.110395  80.000000   
5  2014-01-01    Cotton     Sandy     6.75    14.826739  80.000000   
7  2014-01-01    Tomato      Clay     6.25    18.323272  80.000000   

   Wind_Speed     N     P     K  Crop_Yield  Soil_Quality  
1    8.591577  84.0  66.0  50.0  104.871310     66.666667  
3    2.682683  50.0  40.0  30.0   58.939796     35.000000  
4    7.696070  49.5  45.0  38.5   32.970413     22.166667  
5   10.366657  55.0  44.0  36.0   29.356115     39.375000  
7    8.198084  60.0  45.0  40.0   22.221375     42.291667  


In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from category_encoders import LeaveOneOutEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day
    df = df.drop(columns=['Date'])   

# --- Define Features and Target ---
X = df.drop(columns=['Crop_Yield'])   
y = df['Crop_Yield']


# --- Identify Column Types ---
categorical_features = ['Crop_Type', 'Soil_Type']
numerical_features = [col for col in X.columns if col not in categorical_features]


In [30]:
# --- Split Data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
print(categorical_features)

['Crop_Type', 'Soil_Type']


In [33]:
print(numerical_features)

['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality', 'year', 'month', 'day']


In [34]:
# --- Define Preprocessor ---
preprocessor = ColumnTransformer(
    transformers=[
        ('encode', LeaveOneOutEncoder(cols=categorical_features,handle_unknown='value',handle_missing='value'), categorical_features),
        ('scale', MinMaxScaler(), numerical_features)
    ],remainder='drop'
)

In [35]:
# --- Define Pipeline ---
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42,criterion='squared_error'))
])

In [36]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('encode',
                                                  LeaveOneOutEncoder(cols=['Crop_Type',
                                                                           'Soil_Type']),
                                                  ['Crop_Type', 'Soil_Type']),
                                                 ('scale', MinMaxScaler(),
                                                  ['Soil_pH', 'Temperature',
                                                   'Humidity', 'Wind_Speed',
                                                   'N', 'P', 'K',
                                                   'Soil_Quality', 'year',
                                                   'month', 'day'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [40]:
# --- Predict ---
y_pred = pipeline.predict(X_test)

# --- Evaluate ---
r2 = r2_score(y_test, y_pred)
n = X_test.shape[0]
p = X_test.shape[1]
adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# --- Print results ---
print("Random Forest Regression Results:")
print(f"R² Score: {r2:.4f}")
print(f"Adjusted R²: {adj_r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")

# --- Show few predictions ---
results = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print("\nSample Predictions:")
print(results.head())

Random Forest Regression Results:
R² Score: 0.4712
Adjusted R²: 0.4698
MAE: 12.2622
MSE: 267.1562
RMSE: 16.3449

Sample Predictions:
      Actual  Predicted
0  17.603608  20.935138
1  37.286219  33.395373
2  81.836093  52.272386
3  47.802599  33.388392
4  46.212115  41.749023
